Create a n-1 ordering of ROIs such that each ROI at scale n is assigned to that ROI at scale n-1 that it overlaps maximally with. This order will determine the organization of ROI informtation in tables as well as the yet to be developed numbering system for ROIs.

In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib

In [2]:
# Variables
scales = [7, 12, 20, 36, 64, 122, 197, 325, 444, 'atoms']
# Paths
proj_p = '/home/surchs/GDrive/PhD/Paper/Multiscale_Atlas/Partitions/'
temp_t = os.path.join(proj_p, 'scale_{}.nii.gz')
out_p = '/home/surchs/GDrive/PhD/Paper/Multiscale_Atlas/Labels/roi_order_full.csv'

In [3]:
def find_parent(mask, vol):
    """
    Returns the winner take all overlap parent assignment
    together with the percentage of overlap of the child.
    """
    val, count = np.unique(vol[mask], return_counts=True)
    p_id = np.argmax(count)
    parent = val[p_id]
    overlap = np.round((count[p_id]/np.sum(mask))*100,2)
    
    return parent, overlap

In [4]:
# Load the templates
temp = {scale:nib.load(temp_t.format(scale)).get_data()
        for scale in scales}
n_rois = {scale:len(np.unique(temp[scale]))-1 for scale in scales}

In [5]:
# Find the parents
parents = dict()
for sid, scale in enumerate(scales[::-1]):
    if scale == 7:
        continue
    n_roi = n_rois[scale]
    child = scale
    parent = scales[::-1][sid+1]
    # Mask the child by the parents to take care of the removed midline in the rois
    pmask = temp[parent]==0
    ctemp = temp[child]
    ctemp[pmask] = 0
    parents[scale] = [find_parent(ctemp==roi, temp[parent])[0].astype(int) for roi in np.arange(1,n_roi+1)]

In [6]:
# Create the order table from that information
table = list()
for roi in np.arange(1, n_rois[scales[-1]]+1):
    decomp = [roi]
    for scale in scales[::-1][:-1]:
        parent = parents[scale][roi-1]
        decomp.append(parent)
        roi = parent
    table.append(decomp[::-1])

In [8]:
# Turn the table into a dataframe for easier sorting and saving
df = pd.DataFrame(table, columns=['s7', 's12', 's20', 's36', 's64', 's122', 's197','s325','s444', 'sATOM'])
df.sort_values(by=['s7', 's12', 's20', 's36', 's64', 's122', 's197','s325','s444', 'sATOM'], inplace=True)
df.to_csv(out_p, index=False)

In [9]:
df.head()

,s7,s12,s20,s36,s64,s122,s197,s325,s444,sATOM
1023,1,12,4,1,6,48,84,13,101,1024
1029,1,12,4,1,6,48,84,13,101,1030
1021,1,12,4,1,6,48,84,134,218,1022
1022,1,12,4,1,6,48,84,134,218,1023
1084,1,12,4,1,6,48,84,134,218,1085
